# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
cities = pd.read_csv ("../WeatherPy/Outputs/City_Data.csv")
del cities['Unnamed: 0']
cities

,City,Lat,Lng,Temp F,Max Temp F,Humidity,Cloudiness,Wind Speed,Country,Date
0,hermanus,-34.42,19.23,59.00,59.00,80,14,1.01,ZA,1596062209
1,mar del plata,-38.00,-57.56,46.76,48.99,87,0,4.70,AR,1596062454
2,saint-philippe,-21.36,55.77,61.09,66.20,82,7,3.36,RE,1596062468
3,lebu,-37.62,-73.65,45.18,45.18,71,100,2.19,CL,1596062207
4,chokurdakh,70.63,147.92,45.43,45.43,63,75,9.55,RU,1596062165
...,...,...,...,...,...,...,...,...,...,...
582,maragogi,-9.01,-35.22,74.95,74.95,76,84,13.91,BR,1596063076
583,noumea,-22.28,166.46,78.35,80.60,57,55,13.87,NC,1596063076
584,abu samrah,35.30,37.18,79.70,79.70,31,0,4.90,SY,1596063076
585,phan thiet,10.93,108.10,77.94,77.94,88,100,8.81,VN,1596063077


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [13]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities[["Lat", "Lng"]].astype(float)

# Convert Humidity to float and store
humidity = cities["Humidity"].astype(float)

# Create Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
cond1 = cities.loc[cities['Max Temp F']<77]
cond2 = cond1.loc[cond1['Max Temp F']>64]
cond3 = cond2.loc[cond2['Cloudiness']>40]
cond4 = cond3.loc[cond3['Cloudiness']<70]
cond5 = cond4.loc[cond4['Wind Speed']<10]
ideal = cond5.reset_index(drop=True)
ideal

,City,Lat,Lng,Temp F,Max Temp F,Humidity,Cloudiness,Wind Speed,Country,Date
0,avarua,-21.21,-159.78,71.60,71.60,73,42,8.05,CK,1596062471
1,cajibio,2.62,-76.57,64.04,64.04,88,55,0.76,CO,1596062962
2,huinan,42.62,126.26,71.47,71.47,85,50,5.12,CN,1596062979
3,okha,53.57,142.95,69.93,69.93,89,47,4.43,RU,1596062794
4,khani,41.96,42.96,69.80,69.80,94,57,2.24,GE,1596062541
5,yomou,7.56,-9.27,69.19,69.19,98,54,4.50,GN,1596063030
6,spasskoye,44.61,132.80,71.47,71.47,79,48,8.63,RU,1596063031
7,srandakan,-7.94,110.25,70.38,70.38,81,46,4.68,ID,1596063034
8,cap malheureux,-19.98,57.61,71.01,71.01,77,42,4.00,MU,1596062509
9,berberati,4.26,15.79,67.46,67.46,99,48,1.81,CF,1596063060


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = ideal
hotel_df.reset_index()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Temp F,Max Temp F,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,avarua,-21.21,-159.78,71.60,71.60,73,42,8.05,CK,1596062471,
1,cajibio,2.62,-76.57,64.04,64.04,88,55,0.76,CO,1596062962,
2,huinan,42.62,126.26,71.47,71.47,85,50,5.12,CN,1596062979,
3,okha,53.57,142.95,69.93,69.93,89,47,4.43,RU,1596062794,
4,khani,41.96,42.96,69.80,69.80,94,57,2.24,GE,1596062541,
5,yomou,7.56,-9.27,69.19,69.19,98,54,4.50,GN,1596063030,
6,spasskoye,44.61,132.80,71.47,71.47,79,48,8.63,RU,1596063031,
7,srandakan,-7.94,110.25,70.38,70.38,81,46,4.68,ID,1596063034,
8,cap malheureux,-19.98,57.61,71.01,71.01,77,42,4.00,MU,1596062509,
9,berberati,4.26,15.79,67.46,67.46,99,48,1.81,CF,1596063060,


In [16]:
# geocoordinates
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    target_coordinates = f"{lat},{lng}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "lodging"

# set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel  is {results[0]['name']}.")
    
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Sorry, no Hotel Found Nearby... skipping.")
        hotel_df.loc[index, "Hotel Name"] = "No Hotel Found Nearby"
        
hotel_df

Closest hotel  is The Rarotongan Beach Resort & Lagoonarium.
Sorry, no Hotel Found Nearby... skipping.
Sorry, no Hotel Found Nearby... skipping.
Closest hotel  is Gostinitsa "Sakhalin-Sfera".
Closest hotel  is Nino village.
Sorry, no Hotel Found Nearby... skipping.
Closest hotel  is Gostinitsa "Voyazh".
Sorry, no Hotel Found Nearby... skipping.
Closest hotel  is Lagoon Attitude Hotel.
Closest hotel  is Hôtel palace Georges v.
Closest hotel  is Hotel Le Bien Etre.
Closest hotel  is Hotel Eldorado.


,City,Lat,Lng,Temp F,Max Temp F,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,avarua,-21.21,-159.78,71.60,71.60,73,42,8.05,CK,1596062471,The Rarotongan Beach Resort & Lagoonarium
1,cajibio,2.62,-76.57,64.04,64.04,88,55,0.76,CO,1596062962,No Hotel Found Nearby
2,huinan,42.62,126.26,71.47,71.47,85,50,5.12,CN,1596062979,No Hotel Found Nearby
3,okha,53.57,142.95,69.93,69.93,89,47,4.43,RU,1596062794,"Gostinitsa ""Sakhalin-Sfera"""
4,khani,41.96,42.96,69.80,69.80,94,57,2.24,GE,1596062541,Nino village
5,yomou,7.56,-9.27,69.19,69.19,98,54,4.50,GN,1596063030,No Hotel Found Nearby
6,spasskoye,44.61,132.80,71.47,71.47,79,48,8.63,RU,1596063031,"Gostinitsa ""Voyazh"""
7,srandakan,-7.94,110.25,70.38,70.38,81,46,4.68,ID,1596063034,No Hotel Found Nearby
8,cap malheureux,-19.98,57.61,71.01,71.01,77,42,4.00,MU,1596062509,Lagoon Attitude Hotel
9,berberati,4.26,15.79,67.46,67.46,99,48,1.81,CF,1596063060,Hôtel palace Georges v


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row


# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# Convert bank rate to list
hotels = hotel_df["Hotel Name"] + ", " + hotel_df["City"] + ", " + hotel_df["Country"]
hotel_list = hotels.tolist()
#hotel_list

# Create a map using state centroid coordinates to set markers
marker_locations = hotel_df[['Lat', 'Lng']]

# Create a marker_layer using the hotel list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Hotel: {hotel}" for hotel in hotel_list])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))